The purpose of this code is to determine the necessary fluence of our x-rays, molarity of our sample, and thickness of our jet.
We have the following constraints:
The first pulse should excite 10% of the molecules.  The second pulse should have high enough fluence and the molarity should be high enough with small enough thickness to get sufficient signal to noise.

In [1]:
import numpy as np

We start by using some known constants:

In [2]:
FE_ABS_CS_1 = 398.3 # cm^2/g
FE_ABS_CS_2 = 51.08 # cm^2/g 408.1 at edge
H2O_AW = 18 # g/mol
FE_AW = 55.845 # g/mol
NA = 6.022e23 # mol^-1
EV_TO_JOULES = 1.6e-19

Flow constants:

In [3]:
WATER_VISCOSITY = 8.90e-4 # Pa*s N*s/m^2 @ 25C Note @ 20C 1.0016E-3, @ 30C 7.9722E-4
SURFACE_TENSION = 0.07197 # N/m @ 25 C
WATER_DENSITY = 997.0 # kg/m^3 @ 25 C
WE_J = 0.8 # Weber number for jetting > 0.8

Some constants from the experiment:

In [4]:
SPOT_DIAMETER = 120 # nm
PHOTON_1 = 7186 # eV
PHOTON_2 = 7110 # eV
E_BW = 15 # eV
E_RES = 0.2 # eV
PULSE_DURATION = 10 # fs
TUBE_RADIUS = 125e-6 / 2.0 # m
NBINS = 50 # number of energy bins on spectrometer
MAX_NUM_PHOTONS = 1e8
MAX_NUM_PHOTONS_PIXEL = 11000

Get water cross section <br>
ref: https://physics.nist.gov/PhysRefData/XrayMassCoef/ComTab/water.html

In [5]:
f = np.loadtxt('Water_abs.txt')
H2O_ABS = max(np.interp(PHOTON_1/1e6, f[:,0], f[:,1]), np.interp(PHOTON_2/1e6, f[:,0], f[:,1]))
print('H2O crossection ' + str(H2O_ABS) + ' cm^2/g')

H2O crossection 16.72015 cm^2/g


Some parameters to control:

In [6]:
jet_thickness = 100 # um, between 5 um and 50 um
molar_con = 500 # mM (mol/m^3) (mmol/L) can be up to 100 mM
pulse_energy_1 = .5 # uJ
pulse_energy_2 = .5 # uJ
loss_rate = 1

Now start doing some calculations:

In [7]:
import math
focal_area = math.pi * (SPOT_DIAMETER*1e-7 / 2)**2 # cm^2
focal_volume = jet_thickness*1e-4 * focal_area # cm^3

n_iron = NA * molar_con * focal_volume*1e-6
mass_iron = FE_AW / NA # g

n_water = NA * WATER_DENSITY*1000/H2O_AW * focal_volume*1e-6
mass_water = H2O_AW / NA # g

n_photons_1 = pulse_energy_1*1e-6 / EV_TO_JOULES / PHOTON_1
n_photons_2 = pulse_energy_2*1e-6 / EV_TO_JOULES / PHOTON_2

Find the excitation fraction of the iron atoms from the first pulse:

In [8]:
excite_fraction_1 = n_photons_1 * FE_ABS_CS_1 * mass_iron / focal_area
print('The excitation fraction of FeRu from the first pulse is: {:.2f}%'.format(excite_fraction_1*100))
n_photons_abs_1 = excite_fraction_1 * n_iron
print('The number of photons absorbed by FeRu in the first pulse is: {:.1E}'.format(n_photons_abs_1))
print()
H2O_excite_fraction_1 = n_photons_1 * H2O_ABS * mass_water / focal_area
print('The excitation fraction of water from the first pulse is: {:.2f}%'.format(H2O_excite_fraction_1*100))
H2O_n_photons_abs_1 = H2O_excite_fraction_1 * n_water
print('The number of photons absorbed by water in the first pulse is: {:.1E}'.format(H2O_n_photons_abs_1))

The excitation fraction of FeRu from the first pulse is: 14.20%
The number of photons absorbed by FeRu in the first pulse is: 4.8E+07

The excitation fraction of water from the first pulse is: 0.19%
The number of photons absorbed by water in the first pulse is: 7.2E+07


Find the excitation fraction of the iron atoms from the second pulse:

In [9]:
excite_fraction_2 = n_photons_2 * FE_ABS_CS_2 * mass_iron / focal_area
n_photons_abs_2 = excite_fraction_2 * n_iron
print('The absorption rate of the second pulse from FeRu is {:.2f}%'.format(excite_fraction_2*100))
n_photons_abs_2 = excite_fraction_2 * n_iron
print('The number of photons absorbed by FeRu in the first pulse is: {:.1E}'.format(n_photons_abs_2))
abs_fraction_2 = n_photons_abs_2 / n_photons_2
print('The photon absorption fraction of the second pulse is: {:.1E}%'.format(abs_fraction_2*100))
print()
H2O_excite_fraction_2 = n_photons_2 * H2O_ABS * mass_water / focal_area
print('The absorption rate of the second pulse from water is {:.2f}%'.format(H2O_excite_fraction_2*100))
H2O_n_photons_abs_2 = H2O_excite_fraction_2 * n_water
print('The number of photons absorbed by water in the first pulse is: {:.1E}'.format(H2O_n_photons_abs_2))

The absorption rate of the second pulse from FeRu is 1.84%
The number of photons absorbed by FeRu in the first pulse is: 6.3E+06
The photon absorption fraction of the second pulse is: 1.4E+00%

The absorption rate of the second pulse from water is 0.19%
The number of photons absorbed by water in the first pulse is: 7.3E+07


In each energy bin, we expect to see changes of about 20% at our signals.
Let's err on the side of caution and expect a change of 2% out of the excitation rate of our molecules.
Compare this to the necessary number of photons in each bin to see this change in the signal.

In [10]:
nec_n_photons_bin = 1 / (excite_fraction_1 * abs_fraction_2 * 0.002)**2
n_photons_2 = min(n_photons_2, MAX_NUM_PHOTONS / 2)
photons_bin = n_photons_2 / NBINS
print(photons_bin)
averaging_time = nec_n_photons_bin / photons_bin / 120 / 60 # minutes
print('We will need to average for {:.1F} minutes'.format(averaging_time))

fluence_1 = pulse_energy_1*1e-6 / focal_area / (PULSE_DURATION*1e-15) # W/cm^2
fluence_2 = pulse_energy_2*1e-6 / focal_area / (PULSE_DURATION*1e-15) # W/cm^2
print('We will use a fluence of {:.1E} W/cm^2 for the first pulse and {:.1E} W/cm^2 for the second pulse'.format(fluence_1, fluence_2))

1000000.0
We will need to average for 8.5 minutes
We will use a fluence of 4.4E+17 W/cm^2 for the first pulse and 4.4E+17 W/cm^2 for the second pulse


Calculate the flow and necessary volume.

In [11]:
jet_flow = math.sqrt((WE_J * (math.pi**2) * (TUBE_RADIUS**3) * SURFACE_TENSION) / WATER_DENSITY) *1e6*60 # mL/minute
nec_volume = jet_flow * loss_rate *60*12*4*1e-3 # L
nec_mass = molar_con * nec_volume * FE_AW # mg
print(jet_flow)
print('We will need at least {:.1F} liters of solution and {:.0F} milligrams of solute for four shifts'.format(nec_volume, nec_mass))

0.70777352729183
We will need at least 2.0 liters of solution and 56917 milligrams of solute for four shifts


Assuming a 4 fs pulse, double core hole has a small probability under 10^16 W cm^-2 intensity for CO at 350 eV
Scale this by the Iron cross section

In [12]:
intensity_4fs = 2e16 # W/cm^2
carbon_CS = 3.2e4 #cm^2/g
carbon_AW = 12 #g/mol
scaled_4fs_intensity = intensity_4fs * (carbon_CS / FE_ABS_CS_1 * carbon_AW / FE_AW)**2
print('Scaled intensity for a 4 fs pulse: {:.0E} W/cm^2'.format(scaled_4fs_intensity))

Scaled intensity for a 4 fs pulse: 6E+18 W/cm^2


Assuming a 80 fs pulse, double core hole has a small probability under 10^14 W cm^-2 intensity for CO at 350 eV

In [13]:
intensity_80fs = 5e14 # W/cm^2
scaled_80fs_intensity = intensity_80fs * (carbon_CS / FE_ABS_CS_1 * carbon_AW / FE_AW)**2
print('Scaled intensity for a 80 fs pulse: {:.0E} W/cm^2'.format(scaled_80fs_intensity))

Scaled intensity for a 80 fs pulse: 1E+17 W/cm^2
